In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
train_df = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv')
test_df = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv')

In [ ]:
cols_to_drop = ['Alley', 'PoolQC', 'MiscFeature', 'Fence', 'FireplaceQu']
train_df.drop(columns=cols_to_drop, inplace=True)
test_df.drop(columns=cols_to_drop, inplace=True)

In [ ]:
y = train_df['SalePrice']
X = train_df.drop(['SalePrice'], axis=1)

In [ ]:
test_ids = test_df['Id']
X_test = test_df.drop(['Id'], axis=1)
X.drop(['Id'], axis=1, inplace=True)

In [ ]:
combined = pd.concat([X, X_test], axis=0)

In [ ]:
num_cols = combined.select_dtypes(include=['int64', 'float64']).columns
num_imputer = SimpleImputer(strategy='mean')
combined[num_cols] = num_imputer.fit_transform(combined[num_cols])

In [ ]:
cat_cols = combined.select_dtypes(include='object').columns
cat_imputer = SimpleImputer(strategy='most_frequent')
combined[cat_cols] = cat_imputer.fit_transform(combined[cat_cols])

In [ ]:
combined = pd.get_dummies(combined, drop_first=True)

In [ ]:
X = combined.iloc[:len(train_df), :]
X_test = combined.iloc[len(train_df):, :]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
val_preds = model.predict(X_valid)
rmse = mean_squared_error(y_valid, val_preds, squared=False)
print(f"Validation RMSE: {rmse:.2f}")

In [ ]:
test_preds = model.predict(X_test)

In [ ]:
submission = pd.DataFrame({
    'Id': test_ids,
    'SalePrice': test_preds
})
submission.to_csv('submission.csv', index=False)